In [ ]:
import boto3
import botocore
from iterdub import iterdub as ib
from matplotlib import pyplot as plt
import pandas as pd
from pandas.util import hash_pandas_object
import seaborn as sns
from teeplot import teeplot as tp

# get data

In [ ]:
s3_handle = boto3.resource(
    's3',
    region_name="us-east-2",
    config=botocore.config.Config(
        signature_version=botocore.UNSIGNED,
    ),
)
bucket_handle = s3_handle.Bucket('prq49')

series_profiles, = bucket_handle.objects.filter(
    Prefix=f'endeavor=16/thread-profiles/stage=8+what=elaborated/',
)

In [ ]:
df = pd.read_csv(
    f's3://prq49/{series_profiles.key}',
    compression='xz',
)
dfdigest = '{:x}'.format(hash_pandas_object( df ).sum())
dfdigest

In [ ]:
for stint in df['Stint'].unique():
    exec(f'df{stint} = df[ df["Stint"] == {stint} ]')

In [ ]:
df['Elapsed Update Std'] = df.groupby([
    'Series',
    'Stint',
])['Update'].transform('std') / df.groupby([
    'Series',
    'Stint',
])['Update'].transform('mean')

df['Elapsed Update Span'] = (df.groupby([
    'Series',
    'Stint',
])['Update'].transform('max') - df.groupby([
    'Series',
    'Stint',
])['Update'].transform('min')) / df.groupby([
    'Series',
    'Stint',
])['Update'].transform('mean')

# case study series 16005

In [ ]:
tp.tee(
    sns.lineplot,
    data=df[ df['Series'] == 16005 ],
    x='Stint',
    y='Elapsed Update Std',
    teeplot_outattrs={
        'bucket' : ib.dub( df['Bucket'] ),
        'endeavor' : ib.dub( df['Endeavor'] ),
        'transform' : 'identity',    
        '_dfdigest' : dfdigest,
    },
)

In [ ]:
tp.tee(
    sns.lineplot,
    data=df[ df['Series'] == 16005 ],
    x='Stint',
    y='Elapsed Update Span',
    teeplot_outattrs={
        'bucket' : ib.dub( df['Bucket'] ),
        'endeavor' : ib.dub( df['Endeavor'] ),
        'transform' : 'identity',    
        '_dfdigest' : dfdigest,
    },
)

In [ ]:
print('mean update std', df[ df['Series'] == 16005 ]['Elapsed Update Std'].mean())
print('mean update span', df[ df['Series'] == 16005 ]['Elapsed Update Span'].mean())

In [ ]:
print('mean update std', df[ df['Series'] == 16005 ]['Elapsed Update Std'].max())
print('mean update span', df[ df['Series'] == 16005 ]['Elapsed Update Span'].max())